In [ ]:
import torch 
import math 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
x = torch.empty(3, 4)


In [ ]:
print(type(x))

In [ ]:
print(x)

In [ ]:
xd = torch.empty(3, 4, device=device)   

In [ ]:
xd

In [ ]:
a = torch.rand(3, 4) 
display(a)
b = torch.rand(3, 4)
display(b) 
c = torch.rand(3, 4)
display(c)

x = torch.tensor([a, b, c])

note the difference in initialization!

In [ ]:
zeros = torch.zeros(3, 4) 
zeros

In [ ]:
ones = torch.ones(3, 4)
ones

In [ ]:
torch.manual_seed(18423)
random = torch.rand(2, 3) 
random

randomness

In [ ]:
torch.manual_seed(1729)
random1 = torch.rand(2, 3)
print(random1)

random2 = torch.rand(2, 3)
print(random2)

torch.manual_seed(1729)
random3 = torch.rand(2, 3)
print(random3)

random4 = torch.rand(2, 3)
print(random4)

shapes

In [ ]:
x = torch.empty(2,2,3)
x.shape

In [ ]:
x

In [ ]:
empty_like_x = torch.empty_like(x)
empty_like_x.shape

In [ ]:
zeros_like_x = torch.zeros_like(x) 
zeros_like_x.shape

In [ ]:
ones_like_x = torch.ones_like(x)
ones_like_x.shape

In [ ]:
rand_like_x = torch.rand_like(x)
rand_like_x.shape

use specific data

In [ ]:
some_constants  = torch.tensor([[math.pi, 2.71828], [1.61803, 0.0072897]])
some_constants

In [ ]:
some_constants.shape

In [ ]:
some_integers = torch.tensor((11,2,3,5,8, 11, 23, 55))
some_integers

In [ ]:
some_integers.shape

In [ ]:
more_integers = torch.tensor(((2,4,6), (3, 6, 9)))
more_integers

In [ ]:
more_integers.shape

**torch.tensor() creates a copy of the data passed to it!**

datatypes

In [ ]:
a = torch.ones((2,3), dtype=torch.int16) 
b = torch.rand((2,3), dtype=torch.float64) 
c = b.to(torch.int32)

In [ ]:
a

In [ ]:
b

In [ ]:
c

Math and logic with tensors

In [ ]:
ones = torch.zeros(2, 3) + 1 
ones

In [ ]:
twos = torch.ones(2,2) * 2
twos

In [ ]:
threes = (torch.ones(2,2) * 7 - 1) / 2
threes

In [ ]:
fours = twos ** 2

In [ ]:
sqrt2s = twos ** 0.5
sqrt2s

operations between tensors

In [ ]:
powers2 = twos ** torch.tensor([[1, 2], [3, 4]])
powers2

In [ ]:
fives = torch.ones_like(fours) + fours
fives

In [ ]:
dozens = threes * fours
dozens

tensor broadcasting

works like it does in numpy. broadcasting is a shit idea though, these guys should have gone with an index notation like math does

In [ ]:
rand = torch.rand(2, 4)
doubled = rand * torch.ones(1, 4) * 2
display(rand)
display(doubled)

Broadcasting is a way to perform an operation between tensors that have similarities in their shapes. In the example above, the one-row, four-column tensor is multiplied by both rows of the two-row, four-column tensor.

This is an important operation in Deep Learning. The common example is multiplying a tensor of learning weights by a batch of input tensors, applying the operation to each instance in the batch separately, and returning a tensor of identical shape. While this may be true, tainting the mathematical notation with such things never has been a good idea and never will be. 

## Broadcasting rules:

The rules for broadcasting are:

- Each tensor must have at least one dimension - no empty tensors.

- Comparing the dimension sizes of the two tensors, going **from last to first**:

    1) Each dimension must be equal, or

    2) One of the dimensions must be of size 1, or

    3) The dimension does not exist in one of the tensors

Tensors of identical shape, of course, are trivially “broadcastable”, as you saw earlier. 

In [ ]:
a = torch.ones(4, 3, 2)
b = a * torch.rand(    3, 2) # broadcasting according to rule 3: # 3rd & 2nd dims identical to a, dim 1 absent --> broadcast * over dim 0
display(b)

In [ ]:
c = a * torch.rand(  3, 1) # 3rd dim = 1, 2nd dim identical to a dim 1 absent -> broadcast * over dim 0 and dim 2
display(c)

In [ ]:
d = a * torch.rand(   1, 2) # 3rd dim identical to a, 2nd dim = 1 
display(d) 

these things do create new tensors always

## Altering tensors in Place

This can be problematic with respect to cuda. most math functionshave a version with an appended underscore '_' which does the in place version --> like Julia's funcname! convention

In [ ]:
a = torch.tensor([0, math.pi/4, math.pi/2, 3*math.pi/4])
display(a)

In [ ]:
torch.sin(a)

In [ ]:
b = torch.tensor([0, math.pi/ 4, math.pi/2, 3*math.pi/4])
display(b)

In [ ]:
torch.sin_(b)
display(b)

In [ ]:
a = torch.ones(2, 2)
b = torch.rand(2, 2)
display(a)
display(b)

In [ ]:
a.add_(b)
display(a)

In [ ]:
b.mul_(a)
display(a)
display(b)

There is another option for placing the result of a computation in an existing, allocated tensor. Many of the methods and functions we’ve seen so far - including creation methods! - have an out argument that lets you specify a tensor to receive the output. If the out tensor is the correct shape and dtype, this can happen without a new memory allocation:

In [ ]:
a = torch.rand(2,2)
b = torch.rand(2,2)
c = torch.zeros(2,2,)
old_id = id(c)

print(c)
d = torch.matmul(a, b, out=c)
print(c)
print(d)
print(id(c) == old_id)  # should be True, c is reused

torch.rand(2,2, out=c)
print(c)


we can copy shit with `clone()`

In [ ]:
a = torch.ones(2,2,)
b = a.clone()
display(a)
display(b)

assert b is not a  

a[0][1] = 5645
display(a)
display(b)

## Tensors and autograd

tensors generally are the objects that the autograd in torch works on, just like in JAX. however, in torch they are attached to the tensors. 

There is an important thing to be aware of when using ``clone()``. If your source tensor has autograd, enabled then so will the clone

In [ ]:
a = torch.rand(2, 2, requires_grad=True) # autograd on
display(a)

In [ ]:
b = a.clone() # autograd enabled
display(b)

In [ ]:
c = a.detach().clone()  # autograd disabled
display(c)

## Accelerators

In [ ]:
if torch.accelerator.is_available():
    print('We have an accelerator!')
else:
    print('Sorry, CPU only.')

In [ ]:
torch.accelerator.current_accelerator()

In [ ]:
if torch.accelerator.is_available():
    gpu_rand = torch.rand(2, 2, device=torch.accelerator.current_accelerator())
    print(gpu_rand)
else:
    print('Sorry, CPU only.')

In [ ]:
torch.accelerator.device_count()

In [ ]:
my_device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else torch.device('cpu')

In [ ]:
print('Device: {}'.format(my_device))


In [ ]:
x = torch.rand(2, 2, device=my_device)
print(x)

In [ ]:
y = torch.rand(2, 2)
y = y.to(my_device)
display(y)

In [ ]:
x = torch.rand(2, 2)
y = torch.rand(2, 2, device='cuda')
z = x + y  # exception will be thrown